In [137]:
import pandas as pd
import numpy

In [169]:
df = pd.read_excel("/Users/lukapavageau/Library/Mobile Documents/com~apple~CloudDocs/QIF 4A/semestre 8/projetEntreprise/python/fot10_copie.xlsx",usecols="B,F,I,J,s:w")
pd.set_option('display.max_rows', 10)


/Users/lukapavageau/Library/Mobile Documents/com~apple~CloudDocs/QIF 4A/semestre 8/projetEntreprise/python/venv/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [164]:
ds = df.copy()[21:]



In [174]:
for column in ds.columns:
    ds[column] = pd.to_numeric(ds[column], errors='coerce')


ds.fillna(0, inplace=True)



condition = (((ds['Unnamed: 18'] == 0) | ((ds['Unnamed: 18'] <= (ds['Unnamed: 5'] + ds['Unnamed: 9'])) & (ds['Unnamed: 18'] >= (ds['Unnamed: 5'] + ds['Unnamed: 8'])))) &
             ((ds['Unnamed: 19'] == 0) | ((ds['Unnamed: 19'] <= (ds['Unnamed: 5'] + ds['Unnamed: 9'])) & (ds['Unnamed: 19'] >= (ds['Unnamed: 5'] + ds['Unnamed: 8'])))) &
             ((ds['Unnamed: 20'] == 0) | ((ds['Unnamed: 20'] <= (ds['Unnamed: 5'] + ds['Unnamed: 9'])) & (ds['Unnamed: 20'] >= (ds['Unnamed: 5'] + ds['Unnamed: 8'])))) &
             ((ds['Unnamed: 21'] == 0) | ((ds['Unnamed: 21'] <= (ds['Unnamed: 5'] + ds['Unnamed: 9'])) & (ds['Unnamed: 21'] >= (ds['Unnamed: 5'] + ds['Unnamed: 8'])))) &
             ((ds['Unnamed: 22'] == 0) | ((ds['Unnamed: 22'] <= (ds['Unnamed: 5'] + ds['Unnamed: 9'])) & (ds['Unnamed: 22'] >= (ds['Unnamed: 5'] + ds['Unnamed: 8'])))))






dcotes = ds[ds['Unnamed: 1'] > 10]
#dcotes # toutes les côtes
dC = dcotes[condition]
dC # toutes les cotes conformes



/var/folders/zt/_y3rswnx3v38t49zzr_mdpy00000gn/T/ipykernel_74734/281178355.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dC = dcotes[condition]


,Unnamed: 1,Unnamed: 5,Unnamed: 8,Unnamed: 9,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
21,19.0,0.04,-0.04,0.00,0.000,0.000,0.000,0.000,0.000
24,101.0,70.00,-5.00,5.00,69.500,70.200,69.800,70.500,69.900
27,102.0,3.10,0.00,0.25,3.333,0.000,0.000,0.000,0.000
30,102.0,3.10,0.00,0.25,3.320,0.000,0.000,0.000,0.000
33,102.0,3.10,0.00,0.25,3.307,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...
1152,983.0,22.00,-22.00,0.00,12.966,11.864,13.909,14.712,14.563
1155,984.0,4.50,0.00,0.20,4.650,4.650,4.650,4.650,4.650
1158,985.0,3.10,0.00,0.10,3.130,3.140,3.130,3.150,3.140
1161,986.0,0.50,-0.10,0.10,0.480,0.460,0.450,0.470,0.480
